In [1]:
import xlrd   #for reading excel file
import openpyxl #for writing excel file
import pyodbc #for query SQL Server
from datetime import date
import datetime 

def main():
    
    ExcelFileNm = 'C:/Users/jingh/Documents/startingFile.xlsx'
    ResultFileNm = 'C:/Users/jingh/Documents/resultFile.xlsx'
    start_year =2018
    start_month =1
    end_year = 2018
    end_month =6
    # provide all needed parameters
    
    list_query_strings = query_strings (start_year, start_month, end_year, end_month)
    # from year and month, obtain three strings for the pivot in sql command
    
    print(list_query_strings)
 
    Order_Summary(ExcelFileNm,ResultFileNm,list_query_strings)
    #read excel file, query SQL and write result in another excel file
   



def query_strings (start_year, start_month, end_year, end_month):
    start_date = date(start_year, start_month,1)
    end_date= date(end_year, end_month,1)
    list_days = [ start_date + datetime.timedelta(n) for n in range(int ((end_date - start_date).days)+1)]
    list_days = [x for x in list_days if x.day == 1]
    
    list_str = [s.strftime("%Y")+'-'+str(int(s.strftime("%m"))) for s in list_days]
    list_str1 = ['sum(['+s+']) as [' +s+'-order], sum(['+s+'-shipping]) as [' +s+'-shipping]'  for s in list_str]
    list_str2 =[ '['+s+']' for s in list_str]
    list_str3 = [ '['+s+'-shipping]' for s in list_str]
    
    string1 = ','.join(list_str1)
    string2 = ','.join(list_str2)
    string3 = ','.join(list_str3)
    list_strings = [string1,string2,string3]
    return list_strings
    # Python function return one chunk  


def Order_Summary (ExcelFileName,ResultFileName,list_query_str):
 
     workbook = xlrd.open_workbook(ExcelFileName)
     worksheet = workbook.sheet_by_name("Sheet1") # We need to read the data 
     #from the Excel sheet named "Sheet1"
     num_rows = worksheet.nrows #Number of Rows
     num_cols = worksheet.ncols #Number of Columns
     
     
     Common_Account_list = []
     
     for curr_row in range(1, num_rows, 1):    
         Account=worksheet.cell_value(curr_row, 3)   
         Common_Account_list.append(Account)
     #read through column D for a list of account, pay attention to 3
    
     cleanAccount= [int(x) for x in Common_Account_list if str(x) != '']
     
     print(cleanAccount)
        
     addresses = []
     
     for curr_row in range(1, num_rows, 1):    
         Address=worksheet.cell_value(curr_row, 0)   
         addresses.append(Address)
     #read through column A for a list of address, pay attention for 0
         
     
     addresses=[s if s.find(',')==(-1) else s[0:s.find(',')] for i,  s in enumerate(addresses)  ]
     addresses=[s if s.upper().find(' APT')==(-1) else s[0:s.upper().find(' APT')] for i,  s in enumerate(addresses)  ]
     addresses=[s if s.find(' Apt')==(-1) else s[0:s.find(' Apt')] for i,  s in enumerate(addresses)  ]
     addresses=[s if s.upper().find(' UNIT')==(-1) else s[0:s.upper().find(' UNIT')] for i,  s in enumerate(addresses)  ]
     addresses=[s if s.find(' Unit')==(-1) else s[0:s.find(' Unit')] for i,  s in enumerate(addresses)  ]
     addresses=[s if s.find(' #')==(-1) else s[0:s.find(' #')] for i,  s in enumerate(addresses)  ]
     addresses=[s if s.upper().find(' DRIVE')==(-1) else s[0:(s.upper().find(' DRIVE') +3)] for i,  s in enumerate(addresses)  ]
     addresses=[s if s.upper().find(' LANE')==(-1) else s[0:(s.upper().find(' LANE') +2)] for i,  s in enumerate(addresses)  ]
     addresses=[s if s.upper().find(' CIRCLE')==(-1) else s[0:(s.upper().find(' CIRCLE') +2)] for i,  s in enumerate(addresses)  ]
     addresses=[s if s.upper().find(' ROAD')==(-1) else s[0:(s.upper().find(' ROAD') +2)] for i,  s in enumerate(addresses)  ]
     addresses=[s if s.upper().find(' AVENUE')==(-1) else s[0:(s.upper().find(' AVENUE') +2)] for i,  s in enumerate(addresses)  ]
     addresses=[s if s.upper().find(' BOULEVARD')==(-1) else s[0:(s.upper().find(' BOULEVARD') +2)] for i,  s in enumerate(addresses)  ]
     addresses=[s if s.upper().find(' STREET')==(-1) else s[0:(s.upper().find(' STREET') +2)] for i,  s in enumerate(addresses)  ]
     addresses=[s if s.upper().find(' COURT')==(-1) else s[0:(s.upper().find(' COURT') +2)] for i,  s in enumerate(addresses)  ]
     #clean up the address list by removing later part
        
     addresses=list(set(addresses))
     #obtain the unique values in this list
    
    
     str_add = [""" address like '%""" + every_add + """%'"""for every_add in addresses]
     str_add = ' or '.join(str_add)
     #assember the sql command with the addresses
        
     str_list =[str_add]
     print(str_list)
     
     SQL_Command = """
     
     with Pivot_start as
     (
     SELECT 
         [account_id]
        ,CONCAT (order_Year,'-',order_Month) as order_Period
        ,CONCAT (order_Year,'-',order_Month,'-shipping') as order_period_shipping
        ,SUM (amount_dollar) as order_total
        ,SUM(shipping_fee) as shipping_total
     
     FROM [TEST1].[order_table]     WHERE  account_id not in ( %s) and ( %s )
     GROUP BY
         [account_id]
         ,CONCAT (order_Year,'-',order_Month) 
        ,CONCAT (order_Year,'-',order_Month,'-shipping')  
     )
     select   account_id, %s     from Pivot_start 
     pivot      (sum(order_total) for order_period in (%s)) as P1
     pivot      (sum(shipping_total) for order_period_shipping in (%s)) as p2
     
     group by account_id
     
     
     
     
     
     
     """%(','.join('?' * len(cleanAccount)),str_add,list_query_str[0],list_query_str[1],list_query_str[2])
     
     print(SQL_Command)
     
     conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=LAPTOP-S1V0PRKL;'
                      'Database=PythonTest;'
                      'Trusted_Connection=yes;')
 
     cursor = conn.cursor()
     cursor.execute(SQL_Command,cleanAccount)
     
     
     book = openpyxl.load_workbook(ExcelFileName)
     
     
     sheet1 = book.create_sheet('order_summary')
     
     All_data=cursor.fetchall()
     
     
     column_nm = [column[0] for column in cursor.description]
     
     for i, columns in enumerate(column_nm):
         sheet1.cell(row = 1, column = i+1).value = columns
         
     for i, row1 in enumerate(All_data):
         for j, col in enumerate(row1):
             sheet1.cell(row=i+2, column =j+1).value =col
             
     
     conn.close()
     
     book.save(ResultFileName)
     print('done')

    
if __name__ == '__main__':
    main()    

['sum([2018-1]) as [2018-1-order], sum([2018-1-shipping]) as [2018-1-shipping],sum([2018-2]) as [2018-2-order], sum([2018-2-shipping]) as [2018-2-shipping],sum([2018-3]) as [2018-3-order], sum([2018-3-shipping]) as [2018-3-shipping],sum([2018-4]) as [2018-4-order], sum([2018-4-shipping]) as [2018-4-shipping],sum([2018-5]) as [2018-5-order], sum([2018-5-shipping]) as [2018-5-shipping],sum([2018-6]) as [2018-6-order], sum([2018-6-shipping]) as [2018-6-shipping]', '[2018-1],[2018-2],[2018-3],[2018-4],[2018-5],[2018-6]', '[2018-1-shipping],[2018-2-shipping],[2018-3-shipping],[2018-4-shipping],[2018-5-shipping],[2018-6-shipping]']
[286, 148, 203, 57]
[" address like '%112 Tryon Rd%' or  address like '%110 Tryon Rd%' or  address like '%103 Tryon Rd%' or  address like '%105 Tryon Rd%' or  address like '%108 Tryon Rd%' or  address like '%109 Tryon Rd%' or  address like '%111 Tryon Rd%' or  address like '%102 Tryon Rd%' or  address like '%104 Tryon Rd%' or  address like '%101 Tryon Rd%' or  add

     with Pivot_start as
     (
     SELECT 
         [account_id]
        ,CONCAT (order_Year,'-',order_Month) as order_Period
        ,CONCAT (order_Year,'-',order_Month,'-shipping') as order_period_shipping
        ,SUM (amount_dollar) as order_total
        ,SUM(shipping_fee) as shipping_total
     
     FROM [TEST1].[order_table]     WHERE  account_id not in ( ?,?,?,?) and (  address like '%105 Tryon Rd%' or  address like '%110 Tryon Rd%' or  address like '%103 Tryon Rd%' or  address like '%107 Tryon Rd%' or  address like '%108 Tryon Rd%' or  address like '%101 Tryon Rd%' or  address like '%109 Tryon Rd%' or  address like '%100 Tryon Rd%' or  address like '%104 Tryon Rd%' or  address like '%106 Tryon Rd%' or  address like '%102 Tryon Rd%' or  address like '%111 Tryon Rd%' or  address like '%112 Tryon Rd%' or  address like '%113 Tryon Rd%' )
     GROUP BY
         [account_id]
         ,CONCAT (order_Year,'-',order_Month) 
        ,CONCAT (order_Year,'-',order_Month,'-shipping')  
     )
     select   account_id, sum([2018-1]) as [2018-1-order], sum([2018-1-shipping]) as [2018-1-shipping],sum([2018-2]) as [2018-2-order], sum([2018-2-shipping]) as [2018-2-shipping],sum([2018-3]) as [2018-3-order], sum([2018-3-shipping]) as [2018-3-shipping],sum([2018-4]) as [2018-4-order], sum([2018-4-shipping]) as [2018-4-shipping],sum([2018-5]) as [2018-5-order], sum([2018-5-shipping]) as [2018-5-shipping],sum([2018-6]) as [2018-6-order], sum([2018-6-shipping]) as [2018-6-shipping]     from Pivot_start 
     pivot      (sum(order_total) for order_period in ([2018-1],[2018-2],[2018-3],[2018-4],[2018-5],[2018-6])) as P1
     pivot      (sum(shipping_total) for order_period_shipping in ([2018-1-shipping],[2018-2-shipping],[2018-3-shipping],[2018-4-shipping],[2018-5-shipping],[2018-6-shipping])) as p2
     
     group by account_id